# 기본 셋팅

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings

# 모든 경고 메시지 끄기
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumBarunGothic' # 나눔바른고딕 적용하기

In [ ]:
data= pd.read_csv('/content/drive/MyDrive/기상보간iter.csv')
coord= pd.read_csv('/content/drive/MyDrive/지점명_좌표.csv',encoding='CP949')
peach=pd.read_csv('/content/drive/MyDrive/개화시기)복숭아1차전처리데이터.csv')
pear = pd.read_csv('/content/drive/MyDrive/개화시기)배나무1차전처리데이터.csv')

peach.rename(columns={'년도':'연'},inplace=True)
pear.rename(columns={'년도':'연'},inplace=True)

data['연']=data['일시'].str[:4].astype('int')
data['월']=data['일시'].str[5:7].astype('int')
data['일']=data['일시'].str[8:].astype('int')

data = pd.merge(data, coord, on='지점명', how='left')
data=data.dropna()
data=data[data['월'].isin([2,3])]

data['일교차']=data['최고기온(°C)']-data['최저기온(°C)']


# 필요한 변수 -> 논문 참고
- 위도 경도
- 고도
- 개화일
- 2월 평균 기온 , 최저 기온, 최고 기온
- 3월 평균 기온 , 최저 기온, 최고 기온
- 2,3월 총 강수량
- 2,3월 일조시간 총합


In [ ]:
# 'stata' 데이터프레임을 생성

stata = data.groupby(['연', '월', '지점명', '위도', '경도', '고도', '해양']).agg({
    '최고기온(°C)': ['min','max','mean'],       # 최고 기온의 누적합
    '평균기온(°C)': ['min','max','mean'],       # 평균 기온의 누적합
    '최저기온(°C)': ['min','max','mean'],      # 최저 기온의 누적합
    '일강수량(mm)': 'sum',
    '합계 일조시간(hr)':'sum' # 일교차 기온의 평균
}).reset_index()

"""
stata.columns

Index(['연', '월', '지점명', '위도', '경도', '고도', '해양', '최고기온(°C)_min', '최고기온(°C)_max',
       '최고기온(°C)_mean', '평균기온(°C)_min', '평균기온(°C)_max', '평균기온(°C)_mean',
       '최저기온(°C)_min', '최저기온(°C)_max', '최저기온(°C)_mean', '일강수량(mm)_sum',
       '합계 일조시간(hr)_sum'],
      dtype='object')
"""




"\nstata.columns\n\nIndex(['연', '월', '지점명', '위도', '경도', '고도', '해양', '최고기온(°C)_min', '최고기온(°C)_max',\n       '최고기온(°C)_mean', '평균기온(°C)_min', '평균기온(°C)_max', '평균기온(°C)_mean',\n       '최저기온(°C)_min', '최저기온(°C)_max', '최저기온(°C)_mean', '일강수량(mm)_sum',\n       '합계 일조시간(hr)_sum'],\n      dtype='object')\n"

In [ ]:
# 열 이름을 업데이트
stata.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in stata.columns]

In [ ]:
# '연'과 '지점'을 기준으로 pivot_table을 사용하여 '월' 정보를 와이드 포맷으로 변경
values= ['최고기온(°C)_min', '최고기온(°C)_max',
       '최고기온(°C)_mean', '평균기온(°C)_min', '평균기온(°C)_max', '평균기온(°C)_mean',
       '최저기온(°C)_min', '최저기온(°C)_max', '최저기온(°C)_mean', '일강수량(mm)_sum',
       '합계 일조시간(hr)_sum']

wide_format = stata.pivot_table(index=['연', '지점명','위도','경도','고도','해양'],columns='월', values=values)
# '연'과 '지점명'을 인덱스에서 열로 변환
wide_format.reset_index(inplace=True)

# 열 이름을 원하는 형식으로 변경
wide_format.columns = [f'{str(col[1])}_{col[0]}' if col[1] else col[0] for col in wide_format.columns]

In [ ]:
# '지점명'을 기준으로 정렬하고 '연'에 따라 오름차순 정렬
wide_format = wide_format.sort_values(by=['지점명', '연'], ascending=[True, True])

# 결과를 확인
wide_format.reset_index(inplace=True)
wide_format.dropna(inplace=True)

In [ ]:
y_data = pd.read_csv('/content/drive/MyDrive/서리)1차전처리데이터.csv')
x_data = wide_format.copy()

y_data=y_data.drop('년도',axis=1)
x_data.rename(columns= {'지점명': '지점'}, inplace= True)

y_data['연']=y_data['서리끝'].str[:4].astype('int')


x_data['연'].astype('float')
x_data.dropna(inplace=True)

data =  pd.merge(y_data, x_data, on=['연','지점'], how='left')
data.dropna(inplace=True)
#data.to_csv('개화시기x변수.csv', encoding='cp949', index=False)
data.rename(columns = {"365": "서리365"}, inplace=True)

In [ ]:
peach_data = pd.merge(peach, data, on=['연','지점'], how='left')
pear_data = pd.merge(pear, data, on=['연','지점'], how='left')

peach_data.dropna(inplace=True)
pear_data.dropna(inplace=True)

In [ ]:
peach_data.to_csv('복숭아 개화 데이터.csv', encoding='cp949', index=False)
pear_data.to_csv('배 개화 데이터.csv', encoding='cp949', index=False)